In [1]:
import sys
sys.path.append('../')
import nexullance.LP_optimal as LP_optimal
# import nexullance.LP_gurobi as LP_MP
import topologies.DDF as DDF
import globals as gl
import time
import tracemalloc
from statistics import mean
import networkx as nx
import pynauty as nauty

In [2]:
config = (36, 5)
topo_name='ddf'
EPR=(config[1]+1)//3
_network = DDF.DDFtopo(config[0], config[1])
V=config[0]
assert(V==_network.nx_graph.number_of_nodes())
network_edges=list(_network.nx_graph.edges())

In [3]:
p2p_TM=gl.generate_shift_traffic_pattern(config[0], EPR)
R2R_TM=gl.convert_p2p_traffic_matrix_to_R2R(p2p_TM, config[0], EPR)

In [4]:
#initialize adjacency dict to that of G_1
G_adjacency_dict={n: list(nbrdict.keys()) for n, nbrdict in _network.nx_graph.adjacency()}
vertex_weight={} #first a dictionary of vertex weights, later converted to a list of sets
G_2_edges=0
for i in range(V):
    for j in range(V):
        if R2R_TM[i][j] > 0:
            new_v_id=V+G_2_edges
            G_adjacency_dict[new_v_id]=[i]
            G_adjacency_dict[j].append(new_v_id)
            if R2R_TM[i][j] in vertex_weight.keys():
                assert(new_v_id not in vertex_weight[R2R_TM[i][j]])
                vertex_weight[R2R_TM[i][j]].append(new_v_id)
            else:
                vertex_weight[R2R_TM[i][j]]=[new_v_id]
            G_2_edges+=1

# convert vertex weight dict to a list of color sets
G_color_sets=[]
for color_set in vertex_weight.values():
    G_color_sets.append(set(color_set))

In [5]:
G=nauty.Graph(V+G_2_edges, directed=True, adjacency_dict=G_adjacency_dict, vertex_coloring=G_color_sets)

In [6]:
gen_set=gl.nauty_autgrp_verbose(G, _verbose=True)

Generators of the automorphism group:
[[35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36]]
Size of the generator set: 1
Size (order) of the automorphism group: 2.0
Orbits of the vertices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36]
Number of orbits: 36


In [7]:
# apply automorphism on a vector
def apply_automorphism(vector, automorphism):
    new_vector=[]
    for v in vector:
        new_vector.append(automorphism[v])
    return new_vector
        

# find representative vector by the automorphism group
def representative_vector(vectors, generator_set):
    # inputs: 
    # 1. vectors: a list of vectors of the same size '_n', each vector is in the space of V^n, V is the number of vertices in G_1
    # 2. generator: generator set of the automorphism group
    num_vectors=len(vectors)
    _n=len(vectors[0])

    # outputs:
    # 1. representative vectors
    # 2. the exact automorphisms that maps all vectors to their representatives
    rep_vectors_to_all={} # keys: representative vectors, values: represented vectors

    # initialzing necessary data structures:
    orbit=set()
    checked_vectors=set()

    # start of the algorithm:
    for vector in vectors:
        if tuple(vector) in checked_vectors:
            continue
        representative=tuple(vector)
        rep_vectors_to_all[representative]=[]
        orbit.add(tuple(vector))
        while len(orbit)>0:
            _v=orbit.pop()
            checked_vectors.add(_v)
            for gen in generator_set:
                _w=apply_automorphism(_v, gen)
                if tuple(_w) not in checked_vectors:
                    orbit.add(tuple(_w))
                    rep_vectors_to_all[representative].append(_w)

    return rep_vectors_to_all



In [8]:
vectors=[]
for s in range(V):
    for d in range(V):
        for (i, j) in network_edges:
            vectors.append([s, d, i, j])
            vectors.append([s, d, j, i])

In [9]:
len(vectors)

233280

In [10]:
representative_dict=representative_vector(vectors, gen_set)

In [11]:
len(representative_dict)

116640

In [12]:
len(representative_dict)/len(vectors)

0.5